# XGBRegression (P,A,D coordinates to LMA Features)

In [1]:
"""
Using the identified emotions of each LMA feature set rather than the normal ones
"""

'\nUsing the identified emotions of each LMA feature set rather than the normal ones\n'

## Imports

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.lines as mlines

import tensorflow as tf

import sklearn
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold

import xgboost as xgb

xgb.set_config(verbosity=0)

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import normalize

from sklearn.utils import shuffle
import math
import joblib
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [3]:
model_p = xgb.XGBRegressor(verbosity=0)
model_p.load_model("../../emotion_classifier/model_training/models/bandai_l2p_model.json")

model_a = xgb.XGBRegressor(verbosity=0)
model_a.load_model("../../emotion_classifier/model_training/models/bandai_l2a_model.json")

model_d = xgb.XGBRegressor(verbosity=0)
model_d.load_model("../../emotion_classifier/model_training/models/bandai_l2d_model.json")

scaler = joblib.load('../../emotion_classifier/model_training/datasets/scalers/standardizers/S_BANDAI_5frame.pkl') 

## Load Data

In [4]:
dataset = pd.read_csv('datasets/BANDAI_5frame.csv')
dataset.head()

,max_hand_distance,avg_l_hand_hip_distance,avg_r_hand_hip_distance,max_stride_length,avg_l_hand_chest_distance,avg_r_hand_chest_distance,avg_l_elbow_hip_distance,avg_r_elbow_hip_distance,avg_chest_pelvis_distance,avg_neck_chest_distance,...,r_foot_speed,neck_speed,l_hand_acceleration_magnitude,r_hand_acceleration_magnitude,l_foot_acceleration_magnitude,r_foot_acceleration_magnitude,neck_acceleration_magnitude,EMOTION_P,EMOTION_A,EMOTION_D
0,0.433596,0.130036,0.335475,0.538619,0.184473,0.310089,0.210679,0.093915,0.236151,0.223894,...,-0.137483,-0.139467,0.092920,0.196129,0.221155,0.137483,0.139467,0.05,-0.05,0.0
1,0.348051,0.145184,0.256784,0.455501,0.197954,0.284215,0.172405,0.104876,0.236151,0.223894,...,-0.072441,-0.095084,0.048935,0.032804,0.070625,0.072096,0.045111,0.05,-0.05,0.0
2,0.320294,0.206306,0.180224,0.380190,0.234406,0.259905,0.133006,0.127385,0.236151,0.223894,...,-0.039016,-0.093423,0.030819,0.043059,0.028182,0.036532,0.014232,0.05,-0.05,0.0
3,0.400389,0.287300,0.136974,0.319861,0.279208,0.239450,0.112632,0.156856,0.236151,0.223894,...,-0.036825,-0.101489,0.044534,0.054382,0.015679,0.012813,0.021506,0.05,-0.05,0.0
4,0.465923,0.356129,0.136171,0.309995,0.313229,0.224432,0.112950,0.186680,0.236151,0.223894,...,-0.054338,-0.117573,0.044670,0.036618,0.020074,0.022012,0.024961,0.05,-0.05,0.0


### Split into Test and Train data

In [5]:
train_dataset = dataset.sample(frac=0.8, random_state=42)
test_dataset = dataset.drop(train_dataset.index)

print("No Training Samples:",train_dataset.shape[0])
print("No Test Samples:",test_dataset.shape[0])

train_dataset = shuffle(train_dataset)
test_dataset = shuffle(test_dataset)

No Training Samples: 62841
No Test Samples: 15710


### Split Features from Targets

In [6]:
train_emotions = pd.concat([train_dataset.pop(x) for x in ['EMOTION_P', 'EMOTION_A', 'EMOTION_D']], axis=1)
train_emotions_OG = train_emotions.copy()

test_emotions = pd.concat([test_dataset.pop(x) for x in ['EMOTION_P', 'EMOTION_A', 'EMOTION_D']], axis=1)
test_emotions_OG = test_emotions.copy()

In [7]:
train_dataset_scaled = scaler.transform(train_dataset)
test_dataset_scaled = scaler.transform(test_dataset)

In [8]:
train_emotions_p = model_p.predict(train_dataset_scaled)
train_emotions_a = model_a.predict(train_dataset_scaled)
train_emotions_d = model_d.predict(train_dataset_scaled)

for i in range(len(train_dataset_scaled)):
    train_emotions.iloc[i] = [train_emotions_p[i], train_emotions_a[i], train_emotions_d[i]]

train_emotions.head()

,EMOTION_P,EMOTION_A,EMOTION_D
75470,0.300813,0.407106,0.607748
39689,0.049890,-0.050947,0.000194
45604,0.399620,0.201913,0.347772
58571,0.099815,-0.748947,-0.250809
63544,-0.099830,-0.595596,-0.139180


In [9]:
train_emotions_OG.head()

,EMOTION_P,EMOTION_A,EMOTION_D
75470,0.30,0.40,0.60
39689,0.05,-0.05,0.00
45604,0.40,0.20,0.35
58571,0.10,-0.75,-0.25
63544,-0.10,-0.60,-0.15


In [10]:
# Remove LMA features whose predictions are too different from the real one
index_for_removal = []
for ind in range(0,train_emotions.shape[0]):
    predicted = np.asarray([train_emotions.iloc[ind]])
    true = np.asarray([train_emotions_OG.iloc[ind]])
    
    #print(predicted)
    #print(true)
    
    mae_errors = mean_absolute_error(true, predicted, multioutput='raw_values')
    #print(mae_errors)
    
    if(mae_errors[0] > 0.2 or mae_errors[1] > 0.2 or mae_errors[2] > 0.2):
        index_for_removal.append(ind)
        continue
    
    mae_error = mean_absolute_error(true, predicted)
    #print(mae_error)
    
    if(mae_error > 0.11):
        index_for_removal.append(ind)
        continue
    
    #print()

#print(index_for_removal)


In [11]:
train_emotions_OG.drop(train_emotions_OG.index[index_for_removal], inplace=True)
print(train_emotions_OG.shape)
train_emotions_OG.head()

(62841, 3)


,EMOTION_P,EMOTION_A,EMOTION_D
75470,0.30,0.40,0.60
39689,0.05,-0.05,0.00
45604,0.40,0.20,0.35
58571,0.10,-0.75,-0.25
63544,-0.10,-0.60,-0.15


In [12]:
train_emotions.drop(train_emotions.index[index_for_removal], inplace=True)
print(train_emotions.shape)
train_emotions.head()

(62841, 3)


,EMOTION_P,EMOTION_A,EMOTION_D
75470,0.300813,0.407106,0.607748
39689,0.049890,-0.050947,0.000194
45604,0.399620,0.201913,0.347772
58571,0.099815,-0.748947,-0.250809
63544,-0.099830,-0.595596,-0.139180


In [13]:
train_dataset.drop(train_dataset.index[index_for_removal], inplace=True)
print(train_dataset.shape)
train_dataset.head()

(62841, 25)


,max_hand_distance,avg_l_hand_hip_distance,avg_r_hand_hip_distance,max_stride_length,avg_l_hand_chest_distance,avg_r_hand_chest_distance,avg_l_elbow_hip_distance,avg_r_elbow_hip_distance,avg_chest_pelvis_distance,avg_neck_chest_distance,...,l_hand_speed,r_hand_speed,l_foot_speed,r_foot_speed,neck_speed,l_hand_acceleration_magnitude,r_hand_acceleration_magnitude,l_foot_acceleration_magnitude,r_foot_acceleration_magnitude,neck_acceleration_magnitude
75470,0.367285,0.071199,0.272719,0.528588,0.242614,0.293271,0.127668,0.074848,0.236151,0.223894,...,-0.036255,-0.088657,-0.094759,-0.051019,-0.067651,0.014555,0.013228,0.019040,0.016340,0.008232
39689,0.499315,0.281756,0.268230,0.473666,0.315575,0.261423,0.323470,0.283283,0.236151,0.223894,...,-0.071710,-0.158408,-0.179249,-0.066836,-0.095005,0.049309,0.033290,0.010607,0.026500,0.020169
45604,0.451440,0.381793,0.299557,0.362486,0.303034,0.252990,0.363066,0.311134,0.236151,0.223894,...,-0.189990,-0.137801,-0.082693,-0.204229,-0.140160,0.051703,0.038426,0.026939,0.009845,0.031756
58571,0.599427,0.437627,0.148292,0.481841,0.418800,0.294408,0.172407,0.144091,0.236151,0.223894,...,-0.083338,-0.047953,-0.084298,-0.113480,-0.058145,0.011082,0.007008,0.014940,0.010576,0.005819
63544,0.642665,0.314173,0.324955,0.441583,0.407384,0.401605,0.280005,0.284220,0.236151,0.223894,...,-0.065943,-0.068451,-0.059727,-0.049066,-0.066113,0.012936,0.007125,0.020789,0.012210,0.006656


In [14]:
test_emotions_p = model_p.predict(test_dataset_scaled)
test_emotions_a = model_a.predict(test_dataset_scaled)
test_emotions_d = model_d.predict(test_dataset_scaled)

for i in range(len(test_dataset_scaled)):
    test_emotions.iloc[i] = [test_emotions_p[i], test_emotions_a[i], test_emotions_d[i]]


test_emotions.head()

,EMOTION_P,EMOTION_A,EMOTION_D
25032,-0.095924,-0.606480,-0.138563
73519,0.799901,0.499237,0.148004
20195,-0.101432,-0.577522,-0.147125
4409,0.089288,0.443466,0.360333
9369,0.292307,0.385515,0.571612


In [15]:
test_emotions_OG.head()

,EMOTION_P,EMOTION_A,EMOTION_D
25032,-0.1,-0.6,-0.15
73519,0.8,0.5,0.15
20195,-0.1,-0.6,-0.15
4409,0.1,0.6,0.40
9369,0.3,0.4,0.60


In [16]:
# Remove LMA features whose predictions are too different from the real one
index_for_removal = []
for ind in range(0,test_emotions.shape[0]):
    predicted = np.asarray([test_emotions.iloc[ind]])
    true = np.asarray([test_emotions_OG.iloc[ind]])
    
    #print(predicted)
    #print(true)
    
    mae_errors = mean_absolute_error(true, predicted, multioutput='raw_values')
    #print(mae_errors)
    
    if(mae_errors[0] > 0.2 or mae_errors[1] > 0.2 or mae_errors[2] > 0.2):
        index_for_removal.append(ind)
        continue
    
    mae_error = mean_absolute_error(true, predicted)
    #print(mae_error)
    
    if(mae_error > 0.11):
        index_for_removal.append(ind)
        continue
    
    #print()

#print(index_for_removal)


In [17]:
test_emotions_OG.drop(test_emotions_OG.index[index_for_removal], inplace=True)
print(test_emotions_OG.shape)
test_emotions_OG.head()

(14134, 3)


,EMOTION_P,EMOTION_A,EMOTION_D
25032,-0.1,-0.6,-0.15
73519,0.8,0.5,0.15
20195,-0.1,-0.6,-0.15
4409,0.1,0.6,0.40
9369,0.3,0.4,0.60


In [18]:
test_emotions.drop(test_emotions.index[index_for_removal], inplace=True)
print(test_emotions.shape)
test_emotions.head()

(14134, 3)


,EMOTION_P,EMOTION_A,EMOTION_D
25032,-0.095924,-0.606480,-0.138563
73519,0.799901,0.499237,0.148004
20195,-0.101432,-0.577522,-0.147125
4409,0.089288,0.443466,0.360333
9369,0.292307,0.385515,0.571612


In [19]:
test_dataset.drop(test_dataset.index[index_for_removal], inplace=True)
print(test_dataset.shape)
test_dataset.head()

(14134, 25)


,max_hand_distance,avg_l_hand_hip_distance,avg_r_hand_hip_distance,max_stride_length,avg_l_hand_chest_distance,avg_r_hand_chest_distance,avg_l_elbow_hip_distance,avg_r_elbow_hip_distance,avg_chest_pelvis_distance,avg_neck_chest_distance,...,l_hand_speed,r_hand_speed,l_foot_speed,r_foot_speed,neck_speed,l_hand_acceleration_magnitude,r_hand_acceleration_magnitude,l_foot_acceleration_magnitude,r_foot_acceleration_magnitude,neck_acceleration_magnitude
25032,0.588190,0.288701,0.302491,0.240503,0.393149,0.394590,0.261079,0.278029,0.236151,0.223894,...,-0.048382,-0.018500,-0.013433,-0.060428,-0.034321,0.003695,0.005392,0.002982,0.011216,0.002571
73519,0.413009,0.315935,0.299335,0.188151,0.270281,0.238536,0.295588,0.248602,0.236151,0.223894,...,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
20195,0.248480,0.102389,0.173836,0.408839,0.287581,0.301659,0.124184,0.103176,0.236151,0.223894,...,-0.063585,-0.061677,-0.095469,-0.084127,-0.057761,0.004646,0.005051,0.008298,0.010926,0.003716
4409,0.473798,0.122358,0.449376,0.627396,0.142566,0.372070,0.224453,0.117412,0.236151,0.223894,...,-0.078544,-0.133095,-0.139031,-0.126316,-0.103014,0.009484,0.026278,0.049296,0.013788,0.013618
9369,0.387699,0.238926,0.194126,0.748721,0.233808,0.225606,0.042537,0.241899,0.236151,0.223894,...,-0.182873,-0.161228,-0.232679,-0.155447,-0.169355,0.028096,0.022260,0.012082,0.044820,0.020004


In [20]:
train_y_sets = []
test_y_sets = []
# List of np.arrays each containing the LMA features of a given column (i.e one list per feature)

In [21]:
for colname in train_dataset.columns:
    train_y_sets.append(pd.concat([train_dataset.pop(x) for x in [colname]], axis=1))
    
for colname in test_dataset.columns:
    test_y_sets.append(pd.concat([test_dataset.pop(x) for x in [colname]], axis=1))

In [22]:
for i in range(len(train_y_sets)):
    print(train_y_sets[i].columns[0])

max_hand_distance
avg_l_hand_hip_distance
avg_r_hand_hip_distance
max_stride_length
avg_l_hand_chest_distance
avg_r_hand_chest_distance
avg_l_elbow_hip_distance
avg_r_elbow_hip_distance
avg_chest_pelvis_distance
avg_neck_chest_distance
avg_total_body_volume
avg_lower_body_volume
avg_upper_body_volume
avg_triangle_area_hands_neck
avg_triangle_area_feet_hips
l_hand_speed
r_hand_speed
l_foot_speed
r_foot_speed
neck_speed
l_hand_acceleration_magnitude
r_hand_acceleration_magnitude
l_foot_acceleration_magnitude
r_foot_acceleration_magnitude
neck_acceleration_magnitude


## Define Model

In [91]:
xgbr = xgb.XGBRegressor(verbosity=1) 

params = {
        'eta': [0.01, 0.05, 0.1],
        'min_child_weight': [1, 5, 11, 21],
        'max_depth': [3, 6, 10, 15],
        'gamma': [0, 0.001, 0.01],
        'subsample': [0.75, 1],
        'colsample_bytree': [0.75, 1],
        'lambda': [1, 1.25],
        'alpha': [0.0, 0.25]
        }

models = []

for i in range(len(train_y_sets)):
    best_regressor = random_searches[i].best_estimator_
    params = best_regressor.get_params()
    models.append(
        xgb.XGBRegressor(
                    n_estimators=7500, learning_rate=0.1, max_depth=10, min_child_weight=11, 
                    reg_alpha=params['alpha'], reg_lambda=params['lambda'], gamma=0.0,
                    subsample=0.8, colsample_bytree=0.8, objective="reg:squarederror",
                    tree_method='gpu_hist', gpu_id = 0
                )
    )

## Train Model

In [92]:
for i in range(len(models)):
    print(train_y_sets[i].columns[0])
    models[i].fit(train_emotions, train_y_sets[i])

max_hand_distance
avg_l_hand_hip_distance
avg_r_hand_hip_distance
max_stride_length
avg_l_hand_chest_distance
avg_r_hand_chest_distance
avg_l_elbow_hip_distance
avg_r_elbow_hip_distance
avg_chest_pelvis_distance
avg_neck_chest_distance
avg_total_body_volume
avg_lower_body_volume
avg_upper_body_volume
avg_triangle_area_hands_neck
avg_triangle_area_feet_hips
l_hand_speed
r_hand_speed
l_foot_speed
r_foot_speed
neck_speed
l_hand_acceleration_magnitude
r_hand_acceleration_magnitude
l_foot_acceleration_magnitude
r_foot_acceleration_magnitude
neck_acceleration_magnitude


## Test Model

### Training Score

In [93]:
for i in range(len(models)):
    score = models[i].score(train_emotions, train_y_sets[i])
    print(train_y_sets[i].columns[0])
    print("Training score: ", score)

    print()

max_hand_distance
Training score:  0.6214753924456439

avg_l_hand_hip_distance
Training score:  0.5568360296860405

avg_r_hand_hip_distance
Training score:  0.5442879469727366

max_stride_length
Training score:  0.6297779505938103

avg_l_hand_chest_distance
Training score:  0.519825521167995

avg_r_hand_chest_distance
Training score:  0.5103696579124488

avg_l_elbow_hip_distance
Training score:  0.5419324774124616

avg_r_elbow_hip_distance
Training score:  0.5439429066638551

avg_chest_pelvis_distance
Training score:  -13.246584817267586

avg_neck_chest_distance
Training score:  -3.286601249514768

avg_total_body_volume
Training score:  0.6314267694020335

avg_lower_body_volume
Training score:  0.47405368796724434

avg_upper_body_volume
Training score:  0.4784881066951979

avg_triangle_area_hands_neck
Training score:  0.44899992304475156

avg_triangle_area_feet_hips
Training score:  0.4784086008698605

l_hand_speed
Training score:  0.5595896369501161

r_hand_speed
Training score:  0.55

### Test Set MAE & MSE

In [94]:
for i in range(len(models)):
    pred_y = models[i].predict(test_emotions)
    mse = mean_squared_error(test_y_sets[i], pred_y)
    mae = mean_absolute_error(test_y_sets[i], pred_y)
    print(test_y_sets[i].columns[0])
    print("MSE: %.2f" % mse)
    print("MAE: %.2f" % mae)
    print("Example: ", test_y_sets[i].iloc[0][0])
    print()

max_hand_distance
MSE: 0.05
MAE: 0.17
Example:  0.3763333138864815

avg_l_hand_hip_distance
MSE: 0.02
MAE: 0.11
Example:  0.2926846572464082

avg_r_hand_hip_distance
MSE: 0.02
MAE: 0.11
Example:  0.0896176760950923

max_stride_length
MSE: 0.04
MAE: 0.15
Example:  0.6585657736180796

avg_l_hand_chest_distance
MSE: 0.01
MAE: 0.08
Example:  0.3099358634967842

avg_r_hand_chest_distance
MSE: 0.01
MAE: 0.07
Example:  0.1911176278853235

avg_l_elbow_hip_distance
MSE: 0.01
MAE: 0.09
Example:  0.1624431533611023

avg_r_elbow_hip_distance
MSE: 0.01
MAE: 0.08
Example:  0.1420607085503194

avg_chest_pelvis_distance
MSE: 0.00
MAE: 0.00
Example:  0.236151023637866

avg_neck_chest_distance
MSE: 0.00
MAE: 0.00
Example:  0.2238939937654734

avg_total_body_volume
MSE: 0.03
MAE: 0.13
Example:  0.265028890772126

avg_lower_body_volume
MSE: 0.00
MAE: 0.03
Example:  0.0541787250832223

avg_upper_body_volume
MSE: 0.00
MAE: 0.02
Example:  0.0956981633774018

avg_triangle_area_hands_neck
MSE: 0.00
MAE: 0.04
E

In [95]:
generated_features = []

for i in range(len(models)):
    pred_y = models[i].predict(test_emotions)
    generated_features.append(pred_y)
    
    
rows = []
for i in range(len(generated_features[0])):
    row = []
    for j in range(len(models)):
        row.append(generated_features[j][i])
    rows.append(row)
    
print(len(rows))
        
generated = pd.DataFrame(rows, columns=["max_hand_distance",
          "avg_l_hand_hip_distance",
          "avg_r_hand_hip_distance",
          "max_stride_length",
          "avg_l_hand_chest_distance",
          "avg_r_hand_chest_distance",
          "avg_l_elbow_hip_distance",
          "avg_r_elbow_hip_distance",
          "avg_chest_pelvis_distance",
          "avg_neck_chest_distance",
          
          "avg_total_body_volume",
          "avg_lower_body_volume",
          "avg_upper_body_volume",
          
          "avg_triangle_area_hands_neck",
          "avg_triangle_area_feet_hips",
          
          "l_hand_speed",
          "r_hand_speed",
          "l_foot_speed",
          "r_foot_speed",
          "neck_speed",
          
          "l_hand_acceleration_magnitude",
          "r_hand_acceleration_magnitude",
          "l_foot_acceleration_magnitude",
          "r_foot_acceleration_magnitude",
          "neck_acceleration_magnitude",
    ])

generated.head()

14134


,max_hand_distance,avg_l_hand_hip_distance,avg_r_hand_hip_distance,max_stride_length,avg_l_hand_chest_distance,avg_r_hand_chest_distance,avg_l_elbow_hip_distance,avg_r_elbow_hip_distance,avg_chest_pelvis_distance,avg_neck_chest_distance,...,l_hand_speed,r_hand_speed,l_foot_speed,r_foot_speed,neck_speed,l_hand_acceleration_magnitude,r_hand_acceleration_magnitude,l_foot_acceleration_magnitude,r_foot_acceleration_magnitude,neck_acceleration_magnitude
0,0.333681,0.213049,0.145347,0.292606,0.369152,0.320812,0.208073,0.217122,0.236151,0.223894,...,-0.133928,-0.106063,-0.108382,-0.151641,-0.115605,0.025555,0.020633,0.015105,0.021000,0.011090
1,0.715040,0.262736,0.480391,0.224319,0.339251,0.435358,0.261624,0.276711,0.236151,0.223894,...,-0.029502,-0.072512,-0.084719,-0.009689,-0.049847,0.023612,0.021960,0.031757,0.024601,0.018342
2,0.407281,0.192880,0.315801,0.419170,0.304302,0.372767,0.174932,0.229247,0.236151,0.223894,...,-0.172210,-0.136389,-0.129343,-0.127228,-0.134917,0.027157,0.033100,0.021983,0.019404,0.013025
3,0.827226,0.409430,0.356620,0.637025,0.384151,0.376020,0.245422,0.282281,0.236151,0.223894,...,-0.190814,-0.163834,-0.216858,-0.171716,-0.163555,0.040472,0.033263,0.023679,0.033336,0.017535
4,0.487018,0.291795,0.286611,0.461082,0.325268,0.320134,0.212759,0.204893,0.236151,0.223894,...,-0.102321,-0.088934,-0.105026,-0.099154,-0.089068,0.019718,0.014490,0.019070,0.020464,0.008789


In [96]:
generated_scaled = scaler.transform(generated)

gen_emotions_p = model_p.predict(generated_scaled)
gen_emotions_a = model_a.predict(generated_scaled)
gen_emotions_d = model_d.predict(generated_scaled)

rows = []
for i in range(len(generated_scaled)):
    rows.append([gen_emotions_p[i], gen_emotions_a[i], gen_emotions_d[i]])

gen_emotions = pd.DataFrame(rows, columns=[
            "EMOTION_P", "EMOTION_A", "EMOTION_D"
         ])

gen_emotions.head()

,EMOTION_P,EMOTION_A,EMOTION_D
0,0.284673,-0.111943,-0.167956
1,-0.056531,-0.282745,-0.068700
2,0.375238,0.083016,0.077047
3,0.426599,0.293815,0.164361
4,0.114931,-0.471984,-0.154842


In [100]:
import random

mae_errors = mean_absolute_error(test_emotions, gen_emotions, multioutput='raw_values')
mse_errors = mean_squared_error(test_emotions, gen_emotions, multioutput='raw_values')

features = ["PLEASURE", "AROUSAL", "DOMINANCE"
         ]

print("Overall MAE: " + str(mean_absolute_error(test_emotions, gen_emotions)))

print()
for i in range(len(mse_errors)):
    print("==" + features[i] + "==")
    print("MSE: %.5f" % mse_errors[i])
    print("MAE: %.5f" % mae_errors[i])
    print()
    
for i in range(30):
    row = random.randint(0, len(test_emotions))

    print("Real: " + str([test_emotions_OG.iloc[row,0], test_emotions_OG.iloc[row,1], test_emotions_OG.iloc[row,2]]))
    print("Predicted: " + str([test_emotions.iloc[row,0], test_emotions.iloc[row,1], test_emotions.iloc[row,2]]))
    print("Generated: " + str([gen_emotions.iloc[row,0], gen_emotions.iloc[row,1], gen_emotions.iloc[row,2]]))
    print()

Overall MAE: 0.2046734004045904

==PLEASURE==
MSE: 0.03883
MAE: 0.14625

==AROUSAL==
MSE: 0.13273
MAE: 0.28357

==DOMINANCE==
MSE: 0.06985
MAE: 0.18420

Real: [0.3, 0.4, 0.6]
Predicted: [0.31094929575920105, 0.40011557936668396, 0.6157459020614624]
Generated: [0.19511102, -0.37120625, -0.15404272]

Real: [0.3, 0.4, 0.6]
Predicted: [0.29173508286476135, 0.38976001739501953, 0.6011726260185242]
Generated: [0.37285864, 0.115645014, 0.2109578]

Real: [0.3, 0.4, 0.6]
Predicted: [0.34582337737083435, 0.3678492605686188, 0.5284528732299805]
Generated: [0.19160819, 0.13530667, 0.027441882]

Real: [-0.6, -0.4, -0.3]
Predicted: [-0.6057093739509583, -0.3920977711677551, -0.31791946291923523]
Generated: [-0.5642922, -0.37643725, -0.30486807]

Real: [0.1, -0.75, -0.25]
Predicted: [0.11010432988405228, -0.737703263759613, -0.22721098363399506]
Generated: [0.1852295, -0.45581698, -0.2027882]

Real: [-0.1, -0.6, -0.15]
Predicted: [-0.10644809156656265, -0.6000655293464661, -0.18470680713653564]
Gener

## Save Models

In [101]:
# save to JSON
for i in range(len(models)):
    models[i].save_model("models/bandai/5frame/xgb/" + test_y_sets[i].columns[0] +".json")

# Hyperparameter Grid Search with XGBoost

In [23]:
from datetime import datetime
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import tensorflow as tf

import sklearn
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_absolute_error


import xgboost as xgb

xgb.set_config(verbosity=2)

from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
import joblib

## Load Data

In [24]:
dataset = pd.read_csv('datasets/BANDAI_5frame.csv')
dataset.head()

,max_hand_distance,avg_l_hand_hip_distance,avg_r_hand_hip_distance,max_stride_length,avg_l_hand_chest_distance,avg_r_hand_chest_distance,avg_l_elbow_hip_distance,avg_r_elbow_hip_distance,avg_chest_pelvis_distance,avg_neck_chest_distance,...,r_foot_speed,neck_speed,l_hand_acceleration_magnitude,r_hand_acceleration_magnitude,l_foot_acceleration_magnitude,r_foot_acceleration_magnitude,neck_acceleration_magnitude,EMOTION_P,EMOTION_A,EMOTION_D
0,0.433596,0.130036,0.335475,0.538619,0.184473,0.310089,0.210679,0.093915,0.236151,0.223894,...,-0.137483,-0.139467,0.092920,0.196129,0.221155,0.137483,0.139467,0.05,-0.05,0.0
1,0.348051,0.145184,0.256784,0.455501,0.197954,0.284215,0.172405,0.104876,0.236151,0.223894,...,-0.072441,-0.095084,0.048935,0.032804,0.070625,0.072096,0.045111,0.05,-0.05,0.0
2,0.320294,0.206306,0.180224,0.380190,0.234406,0.259905,0.133006,0.127385,0.236151,0.223894,...,-0.039016,-0.093423,0.030819,0.043059,0.028182,0.036532,0.014232,0.05,-0.05,0.0
3,0.400389,0.287300,0.136974,0.319861,0.279208,0.239450,0.112632,0.156856,0.236151,0.223894,...,-0.036825,-0.101489,0.044534,0.054382,0.015679,0.012813,0.021506,0.05,-0.05,0.0
4,0.465923,0.356129,0.136171,0.309995,0.313229,0.224432,0.112950,0.186680,0.236151,0.223894,...,-0.054338,-0.117573,0.044670,0.036618,0.020074,0.022012,0.024961,0.05,-0.05,0.0


### Split into Test and Train data

In [25]:
train_dataset = dataset.sample(frac=0.8, random_state=42)
test_dataset = dataset.drop(train_dataset.index)

print("No Training Samples:",train_dataset.shape[0])
print("No Test Samples:",test_dataset.shape[0])

train_dataset = shuffle(train_dataset)
test_dataset = shuffle(test_dataset)

No Training Samples: 62841
No Test Samples: 15710


### Split Features from Targets

In [26]:
train_emotions = pd.concat([train_dataset.pop(x) for x in ['EMOTION_P', 'EMOTION_A', 'EMOTION_D']], axis=1)
train_emotions_OG = train_emotions.copy()

test_emotions = pd.concat([test_dataset.pop(x) for x in ['EMOTION_P', 'EMOTION_A', 'EMOTION_D']], axis=1)
test_emotions_OG = test_emotions.copy()

In [27]:
train_dataset_scaled = scaler.transform(train_dataset)
test_dataset_scaled = scaler.transform(test_dataset)

In [28]:
train_emotions_p = model_p.predict(train_dataset_scaled)
train_emotions_a = model_a.predict(train_dataset_scaled)
train_emotions_d = model_d.predict(train_dataset_scaled)

for i in range(len(train_dataset_scaled)):
    train_emotions.iloc[i] = [train_emotions_p[i], train_emotions_a[i], train_emotions_d[i]]

train_emotions.head()

,EMOTION_P,EMOTION_A,EMOTION_D
57742,0.599882,0.400190,0.095782
12143,0.300002,0.401675,0.598994
36559,-0.100105,-0.603663,-0.154060
42153,-0.100726,-0.600995,-0.152048
74886,-0.099899,-0.600583,-0.150989


In [29]:
train_emotions_OG.head()

,EMOTION_P,EMOTION_A,EMOTION_D
57742,0.6,0.4,0.10
12143,0.3,0.4,0.60
36559,-0.1,-0.6,-0.15
42153,-0.1,-0.6,-0.15
74886,-0.1,-0.6,-0.15


In [30]:
# Remove LMA features whose predictions are too different from the real one
index_for_removal = []
for ind in range(0,train_emotions.shape[0]):
    predicted = np.asarray([train_emotions.iloc[ind]])
    true = np.asarray([train_emotions_OG.iloc[ind]])
    
    #print(predicted)
    #print(true)
    
    mae_errors = mean_absolute_error(true, predicted, multioutput='raw_values')
    #print(mae_errors)
    
    if(mae_errors[0] > 0.2 or mae_errors[1] > 0.2 or mae_errors[2] > 0.2):
        index_for_removal.append(ind)
        continue
    
    mae_error = mean_absolute_error(true, predicted)
    #print(mae_error)
    
    if(mae_error > 0.11):
        index_for_removal.append(ind)
        continue
    
    #print()

#print(index_for_removal)


In [31]:
train_emotions_OG.drop(train_emotions_OG.index[index_for_removal], inplace=True)
print(train_emotions_OG.shape)
train_emotions_OG.head()

(62841, 3)


,EMOTION_P,EMOTION_A,EMOTION_D
57742,0.6,0.4,0.10
12143,0.3,0.4,0.60
36559,-0.1,-0.6,-0.15
42153,-0.1,-0.6,-0.15
74886,-0.1,-0.6,-0.15


In [32]:
train_emotions.drop(train_emotions.index[index_for_removal], inplace=True)
print(train_emotions.shape)
train_emotions.head()

(62841, 3)


,EMOTION_P,EMOTION_A,EMOTION_D
57742,0.599882,0.400190,0.095782
12143,0.300002,0.401675,0.598994
36559,-0.100105,-0.603663,-0.154060
42153,-0.100726,-0.600995,-0.152048
74886,-0.099899,-0.600583,-0.150989


In [33]:
train_dataset.drop(train_dataset.index[index_for_removal], inplace=True)
print(train_dataset.shape)
train_dataset.head()

(62841, 25)


,max_hand_distance,avg_l_hand_hip_distance,avg_r_hand_hip_distance,max_stride_length,avg_l_hand_chest_distance,avg_r_hand_chest_distance,avg_l_elbow_hip_distance,avg_r_elbow_hip_distance,avg_chest_pelvis_distance,avg_neck_chest_distance,...,l_hand_speed,r_hand_speed,l_foot_speed,r_foot_speed,neck_speed,l_hand_acceleration_magnitude,r_hand_acceleration_magnitude,l_foot_acceleration_magnitude,r_foot_acceleration_magnitude,neck_acceleration_magnitude
57742,0.809504,0.420113,0.381539,0.623642,0.468395,0.394640,0.291945,0.269929,0.236151,0.223894,...,-0.143711,-0.074761,-0.073505,-0.133441,-0.088978,0.010469,0.014763,0.005491,0.028007,0.013098
12143,0.817962,0.398630,0.416399,0.324186,0.449560,0.451846,0.341901,0.334180,0.236151,0.223894,...,-0.044646,-0.080660,-0.134852,-0.026424,-0.062779,0.009751,0.008096,0.005025,0.006841,0.010377
36559,0.618099,0.100899,0.594393,0.538244,0.195376,0.456488,0.191703,0.214596,0.236151,0.223894,...,-0.110903,-0.111969,-0.099005,-0.140504,-0.109902,0.020548,0.024496,0.045674,0.006866,0.007170
42153,0.568484,0.546575,0.144221,0.493973,0.430774,0.217647,0.180157,0.186170,0.236151,0.223894,...,-0.089636,-0.105723,-0.100614,-0.054690,-0.086101,0.018972,0.012423,0.007694,0.032313,0.008331
74886,0.521041,0.139863,0.500190,0.423790,0.198303,0.395375,0.195357,0.149781,0.236151,0.223894,...,-0.068379,-0.077513,-0.095812,-0.094615,-0.072494,0.016428,0.020581,0.042009,0.005819,0.011247


In [34]:
test_emotions_p = model_p.predict(test_dataset_scaled)
test_emotions_a = model_a.predict(test_dataset_scaled)
test_emotions_d = model_d.predict(test_dataset_scaled)

for i in range(len(test_dataset_scaled)):
    test_emotions.iloc[i] = [test_emotions_p[i], test_emotions_a[i], test_emotions_d[i]]


test_emotions.head()

,EMOTION_P,EMOTION_A,EMOTION_D
9522,0.541955,0.404174,0.073686
48599,0.075206,-0.065621,0.023219
42269,0.602221,0.397315,0.096417
38207,0.113180,0.543130,0.294451
31932,0.080195,0.361856,0.256567


In [35]:
test_emotions_OG.head()

,EMOTION_P,EMOTION_A,EMOTION_D
9522,0.60,0.40,0.1
48599,0.05,-0.05,0.0
42269,0.60,0.40,0.1
38207,0.10,0.60,0.4
31932,0.05,-0.05,0.0


In [36]:
# Remove LMA features whose predictions are too different from the real one
index_for_removal = []
for ind in range(0,test_emotions.shape[0]):
    predicted = np.asarray([test_emotions.iloc[ind]])
    true = np.asarray([test_emotions_OG.iloc[ind]])
    
    #print(predicted)
    #print(true)
    
    mae_errors = mean_absolute_error(true, predicted, multioutput='raw_values')
    #print(mae_errors)
    
    if(mae_errors[0] > 0.2 or mae_errors[1] > 0.2 or mae_errors[2] > 0.2):
        index_for_removal.append(ind)
        continue
    
    mae_error = mean_absolute_error(true, predicted)
    #print(mae_error)
    
    if(mae_error > 0.11):
        index_for_removal.append(ind)
        continue
    
    #print()

#print(index_for_removal)


In [37]:
test_emotions_OG.drop(test_emotions_OG.index[index_for_removal], inplace=True)
print(test_emotions_OG.shape)
test_emotions_OG.head()

(14134, 3)


,EMOTION_P,EMOTION_A,EMOTION_D
9522,0.60,0.40,0.10
48599,0.05,-0.05,0.00
42269,0.60,0.40,0.10
38207,0.10,0.60,0.40
76170,-0.10,-0.60,-0.15


In [38]:
test_emotions.drop(test_emotions.index[index_for_removal], inplace=True)
print(test_emotions.shape)
test_emotions.head()

(14134, 3)


,EMOTION_P,EMOTION_A,EMOTION_D
9522,0.541955,0.404174,0.073686
48599,0.075206,-0.065621,0.023219
42269,0.602221,0.397315,0.096417
38207,0.113180,0.543130,0.294451
76170,-0.071303,-0.578708,-0.132384


In [39]:
test_dataset.drop(test_dataset.index[index_for_removal], inplace=True)
print(test_dataset.shape)
test_dataset.head()

(14134, 25)


,max_hand_distance,avg_l_hand_hip_distance,avg_r_hand_hip_distance,max_stride_length,avg_l_hand_chest_distance,avg_r_hand_chest_distance,avg_l_elbow_hip_distance,avg_r_elbow_hip_distance,avg_chest_pelvis_distance,avg_neck_chest_distance,...,l_hand_speed,r_hand_speed,l_foot_speed,r_foot_speed,neck_speed,l_hand_acceleration_magnitude,r_hand_acceleration_magnitude,l_foot_acceleration_magnitude,r_foot_acceleration_magnitude,neck_acceleration_magnitude
9522,0.376333,0.292685,0.089618,0.658566,0.309936,0.191118,0.162443,0.142061,0.236151,0.223894,...,-0.116808,-0.229898,-0.226500,-0.106399,-0.160896,0.020381,0.019865,0.043979,0.019786,0.012636
48599,0.411498,0.099391,0.449231,0.436756,0.161160,0.368629,0.163027,0.136584,0.236151,0.223894,...,-0.215469,-0.195944,-0.127079,-0.236523,-0.195251,0.021007,0.010435,0.050059,0.025921,0.009353
42269,1.012155,0.692504,0.402306,0.510866,0.628199,0.406106,0.421180,0.334610,0.236151,0.223894,...,-0.187017,-0.224813,-0.194634,-0.146936,-0.177708,0.039914,0.021178,0.018063,0.039256,0.006622
38207,0.428599,0.367135,0.136222,0.492338,0.326365,0.251081,0.121335,0.103004,0.236151,0.223894,...,-0.090358,-0.153597,-0.110357,-0.060135,-0.075055,0.028808,0.068298,0.038232,0.061528,0.012757
76170,0.467636,0.289947,0.331012,0.346103,0.300093,0.281826,0.291987,0.278736,0.236151,0.223894,...,-0.117331,-0.079672,-0.045519,-0.097064,-0.081729,0.009169,0.012452,0.009270,0.010650,0.005896


In [40]:
train_y_sets = []
test_y_sets = []
# List of np.arrays each containing the LMA features of a given column (i.e one list per feature)

In [41]:
for colname in train_dataset.columns:
    train_y_sets.append(pd.concat([train_dataset.pop(x) for x in [colname]], axis=1))
    
for colname in test_dataset.columns:
    test_y_sets.append(pd.concat([test_dataset.pop(x) for x in [colname]], axis=1))

In [42]:
for i in range(len(train_y_sets)):
    print(train_y_sets[i].columns[0])

max_hand_distance
avg_l_hand_hip_distance
avg_r_hand_hip_distance
max_stride_length
avg_l_hand_chest_distance
avg_r_hand_chest_distance
avg_l_elbow_hip_distance
avg_r_elbow_hip_distance
avg_chest_pelvis_distance
avg_neck_chest_distance
avg_total_body_volume
avg_lower_body_volume
avg_upper_body_volume
avg_triangle_area_hands_neck
avg_triangle_area_feet_hips
l_hand_speed
r_hand_speed
l_foot_speed
r_foot_speed
neck_speed
l_hand_acceleration_magnitude
r_hand_acceleration_magnitude
l_foot_acceleration_magnitude
r_foot_acceleration_magnitude
neck_acceleration_magnitude


## RandomSearchCV

In [44]:
# A parameter grid for XGBoost
# https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/
params = {
        'eta': [0.01, 0.05, 0.1],
        'min_child_weight': [1, 5, 11, 21],
        'max_depth': [3, 6, 10, 15],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0],
        'lambda': [0.85, 1, 1.25],
        'alpha': [0.0, 0.1, 0.25]
        }

n_iter = 125

In [45]:
models = []

for i in range(len(train_y_sets)):
    models.append(
        xgb.XGBRegressor(
                    n_estimators=3000, 
                    objective="reg:squarederror",
                    tree_method='gpu_hist',
                    gpu_id=1
                )
    )

In [46]:
# Pleasure
# run randomized search

random_searches = []

for i in range(5):
    print(train_y_sets[i].columns[0])
    print(i)
    
    kfold = KFold(n_splits=2, shuffle=True)

    random_search = RandomizedSearchCV(models[i], param_distributions=params,
                               cv=kfold, scoring='neg_mean_squared_error', n_iter = n_iter)

    start = time.time()
    random_search.fit(train_emotions, train_y_sets[i])
    
    random_searches.append(random_search)

    print("GridSearchCV took %.2f seconds"
          " parameter settings." % ((time.time() - start)))
    
    print()

max_hand_distance
0
GridSearchCV took 9366.06 seconds parameter settings.

avg_l_hand_hip_distance
1
GridSearchCV took 7985.97 seconds parameter settings.

avg_r_hand_hip_distance
2
GridSearchCV took 7454.45 seconds parameter settings.

max_stride_length
3
GridSearchCV took 7826.54 seconds parameter settings.

avg_l_hand_chest_distance
4
GridSearchCV took 6680.14 seconds parameter settings.



In [47]:
for i in range(5,10):
    print(train_y_sets[i].columns[0])
    print(i)
    
    kfold = KFold(n_splits=2, shuffle=True)

    random_search = RandomizedSearchCV(models[i], param_distributions=params,
                               cv=kfold, scoring='neg_mean_squared_error', n_iter = n_iter)

    start = time.time()
    random_search.fit(train_emotions, train_y_sets[i])
    
    random_searches.append(random_search)

    print("GridSearchCV took %.2f seconds"
          " parameter settings." % ((time.time() - start)))
    
    print()

avg_r_hand_chest_distance
5
GridSearchCV took 6571.84 seconds parameter settings.

avg_l_elbow_hip_distance
6
GridSearchCV took 7442.95 seconds parameter settings.

avg_r_elbow_hip_distance
7
GridSearchCV took 7031.78 seconds parameter settings.

avg_chest_pelvis_distance
8
GridSearchCV took 376.83 seconds parameter settings.

avg_neck_chest_distance
9
GridSearchCV took 381.67 seconds parameter settings.



In [48]:
for i in range(10,15):
    print(train_y_sets[i].columns[0])
    print(i)
    
    kfold = KFold(n_splits=2, shuffle=True)

    random_search = RandomizedSearchCV(models[i], param_distributions=params,
                               cv=kfold, scoring='neg_mean_squared_error', n_iter = n_iter)

    start = time.time()
    random_search.fit(train_emotions, train_y_sets[i])
    
    random_searches.append(random_search)

    print("GridSearchCV took %.2f seconds"
          " parameter settings." % ((time.time() - start)))
    
    print()

avg_total_body_volume
10
GridSearchCV took 8836.56 seconds parameter settings.

avg_lower_body_volume
11
GridSearchCV took 5374.24 seconds parameter settings.

avg_upper_body_volume
12
GridSearchCV took 5265.42 seconds parameter settings.

avg_triangle_area_hands_neck
13
GridSearchCV took 5766.23 seconds parameter settings.

avg_triangle_area_feet_hips
14
GridSearchCV took 6835.30 seconds parameter settings.



In [49]:
for i in range(15,20):
    print(train_y_sets[i].columns[0])
    print(i)
    
    kfold = KFold(n_splits=2, shuffle=True)

    random_search = RandomizedSearchCV(models[i], param_distributions=params,
                               cv=kfold, scoring='neg_mean_squared_error', n_iter = n_iter)

    start = time.time()
    random_search.fit(train_emotions, train_y_sets[i])
    
    random_searches.append(random_search)

    print("GridSearchCV took %.2f seconds"
          " parameter settings." % ((time.time() - start)))
    
    print()

l_hand_speed
15
GridSearchCV took 4886.90 seconds parameter settings.

r_hand_speed
16
GridSearchCV took 5775.62 seconds parameter settings.

l_foot_speed
17
GridSearchCV took 6915.67 seconds parameter settings.

r_foot_speed
18
GridSearchCV took 5554.00 seconds parameter settings.

neck_speed
19
GridSearchCV took 5373.08 seconds parameter settings.



In [50]:
for i in range(20,25):
    print(train_y_sets[i].columns[0])
    print(i)
    
    kfold = KFold(n_splits=2, shuffle=True)

    random_search = RandomizedSearchCV(models[i], param_distributions=params,
                               cv=kfold, scoring='neg_mean_squared_error', n_iter = n_iter)

    start = time.time()
    random_search.fit(train_emotions, train_y_sets[i])
    
    random_searches.append(random_search)

    print("GridSearchCV took %.2f seconds"
          " parameter settings." % ((time.time() - start)))
    
    print()

l_hand_acceleration_magnitude
20
GridSearchCV took 4537.24 seconds parameter settings.

r_hand_acceleration_magnitude
21
GridSearchCV took 3694.65 seconds parameter settings.

l_foot_acceleration_magnitude
22
GridSearchCV took 4394.99 seconds parameter settings.

r_foot_acceleration_magnitude
23
GridSearchCV took 4252.28 seconds parameter settings.

neck_acceleration_magnitude
24
GridSearchCV took 4695.47 seconds parameter settings.



### Results

In [60]:
best_regressors = []
for i in range(len(random_searches)):
    print(train_y_sets[i].columns[0])
    best_regressor = random_searches[i].best_estimator_

    print(best_regressor.get_params())
    best_regressors.append(best_regressor)
    print()

max_hand_distance
{'objective': 'reg:squarederror', 'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 0.8, 'enable_categorical': False, 'gamma': 0, 'gpu_id': 1, 'importance_type': None, 'interaction_constraints': '', 'learning_rate': 0.00999999978, 'max_delta_step': 0, 'max_depth': 6, 'min_child_weight': 1, 'missing': nan, 'monotone_constraints': '()', 'n_estimators': 3000, 'n_jobs': 12, 'num_parallel_tree': 1, 'predictor': 'auto', 'random_state': 0, 'reg_alpha': 0.25, 'reg_lambda': 0.850000024, 'scale_pos_weight': 1, 'subsample': 1.0, 'tree_method': 'gpu_hist', 'validate_parameters': 1, 'verbosity': None, 'lambda': 0.85, 'eta': 0.01, 'alpha': 0.25}

avg_l_hand_hip_distance
{'objective': 'reg:squarederror', 'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 1.0, 'enable_categorical': False, 'gamma': 0, 'gpu_id': 1, 'importance_type': None, 'interaction_constraints': '', 'lea

In [52]:
for i in range(len(best_regressors)):
    pred_y = best_regressors[i].predict(test_emotions)
    mse = mean_squared_error(test_y_sets[i], pred_y)
    mae = mean_absolute_error(test_y_sets[i], pred_y)
    print(test_y_sets[i].columns[0])
    print("MSE: %.2f" % mse)
    print("MAE: %.2f" % mae)
    print("Example: ", test_y_sets[i].iloc[0][0])
    print()

max_hand_distance
MSE: 0.03
MAE: 0.15
Example:  0.3763333138864815

avg_l_hand_hip_distance
MSE: 0.02
MAE: 0.10
Example:  0.2926846572464082

avg_r_hand_hip_distance
MSE: 0.01
MAE: 0.09
Example:  0.0896176760950923

max_stride_length
MSE: 0.03
MAE: 0.13
Example:  0.6585657736180796

avg_l_hand_chest_distance
MSE: 0.01
MAE: 0.07
Example:  0.3099358634967842

avg_r_hand_chest_distance
MSE: 0.01
MAE: 0.07
Example:  0.1911176278853235

avg_l_elbow_hip_distance
MSE: 0.01
MAE: 0.08
Example:  0.1624431533611023

avg_r_elbow_hip_distance
MSE: 0.01
MAE: 0.07
Example:  0.1420607085503194

avg_chest_pelvis_distance
MSE: 0.00
MAE: 0.00
Example:  0.236151023637866

avg_neck_chest_distance
MSE: 0.00
MAE: 0.00
Example:  0.2238939937654734

avg_total_body_volume
MSE: 0.02
MAE: 0.11
Example:  0.265028890772126

avg_lower_body_volume
MSE: 0.00
MAE: 0.03
Example:  0.0541787250832223

avg_upper_body_volume
MSE: 0.00
MAE: 0.02
Example:  0.0956981633774018

avg_triangle_area_hands_neck
MSE: 0.00
MAE: 0.03
E

In [53]:
generated_features = []

for i in range(len(best_regressors)):
    pred_y = best_regressors[i].predict(test_emotions)
    generated_features.append(pred_y)
    
    
rows = []
for i in range(len(generated_features[0])):
    row = []
    for j in range(len(best_regressors)):
        row.append(generated_features[j][i])
    rows.append(row)
    
print(len(rows))
        
generated = pd.DataFrame(rows, columns=["max_hand_distance",
          "avg_l_hand_hip_distance",
          "avg_r_hand_hip_distance",
          "max_stride_length",
          "avg_l_hand_chest_distance",
          "avg_r_hand_chest_distance",
          "avg_l_elbow_hip_distance",
          "avg_r_elbow_hip_distance",
          "avg_chest_pelvis_distance",
          "avg_neck_chest_distance",
          
          "avg_total_body_volume",
          "avg_lower_body_volume",
          "avg_upper_body_volume",
          
          "avg_triangle_area_hands_neck",
          "avg_triangle_area_feet_hips",
          
          "l_hand_speed",
          "r_hand_speed",
          "l_foot_speed",
          "r_foot_speed",
          "neck_speed",
          
          "l_hand_acceleration_magnitude",
          "r_hand_acceleration_magnitude",
          "l_foot_acceleration_magnitude",
          "r_foot_acceleration_magnitude",
          "neck_acceleration_magnitude",
    ])

generated.head()

14134


,max_hand_distance,avg_l_hand_hip_distance,avg_r_hand_hip_distance,max_stride_length,avg_l_hand_chest_distance,avg_r_hand_chest_distance,avg_l_elbow_hip_distance,avg_r_elbow_hip_distance,avg_chest_pelvis_distance,avg_neck_chest_distance,...,l_hand_speed,r_hand_speed,l_foot_speed,r_foot_speed,neck_speed,l_hand_acceleration_magnitude,r_hand_acceleration_magnitude,l_foot_acceleration_magnitude,r_foot_acceleration_magnitude,neck_acceleration_magnitude
0,0.532113,0.362142,0.293393,0.473878,0.372519,0.360185,0.220564,0.198578,0.236151,0.223894,...,-0.119724,-0.116576,-0.116454,-0.114466,-0.107146,0.024443,0.022764,0.019244,0.019991,0.011416
1,0.587141,0.314081,0.320447,0.492085,0.359139,0.342489,0.228726,0.222792,0.236151,0.223894,...,-0.079798,-0.109198,-0.103821,-0.071927,-0.082450,0.016702,0.016841,0.019690,0.021563,0.012482
2,0.626081,0.349547,0.339247,0.492209,0.362786,0.382108,0.223100,0.231308,0.236151,0.223894,...,-0.136180,-0.137619,-0.133187,-0.130424,-0.125184,0.030397,0.030321,0.024132,0.021797,0.014509
3,0.676452,0.389727,0.366884,0.518335,0.363194,0.369228,0.269754,0.283954,0.236151,0.223894,...,-0.138631,-0.133573,-0.149092,-0.126538,-0.120436,0.036591,0.038462,0.031724,0.026855,0.019107
4,0.482607,0.275913,0.283407,0.418134,0.329847,0.330527,0.206376,0.203298,0.236151,0.223894,...,-0.089317,-0.084167,-0.085068,-0.086110,-0.078881,0.016940,0.013095,0.016608,0.016146,0.008744


In [54]:
generated_scaled = scaler.transform(generated)

gen_emotions_p = model_p.predict(generated_scaled)
gen_emotions_a = model_a.predict(generated_scaled)
gen_emotions_d = model_d.predict(generated_scaled)

rows = []
for i in range(len(generated_scaled)):
    rows.append([gen_emotions_p[i], gen_emotions_a[i], gen_emotions_d[i]])

gen_emotions = pd.DataFrame(rows, columns=[
            "EMOTION_P", "EMOTION_A", "EMOTION_D"
         ])

gen_emotions.head()

,EMOTION_P,EMOTION_A,EMOTION_D
0,0.245268,-0.273565,-0.146036
1,0.188328,-0.418062,-0.082498
2,0.381424,-0.038246,-0.004399
3,0.057700,-0.029398,0.031987
4,0.070690,-0.479293,-0.160376


In [55]:
import random

mae_errors = mean_absolute_error(test_emotions, gen_emotions, multioutput='raw_values')
mse_errors = mean_squared_error(test_emotions, gen_emotions, multioutput='raw_values')

features = ["PLEASURE", "AROUSAL", "DOMINANCE"
         ]

print("Overall MAE: " + str(mean_absolute_error(test_emotions, gen_emotions)))

print()
for i in range(len(mse_errors)):
    print("==" + features[i] + "==")
    print("MSE: %.5f" % mse_errors[i])
    print("MAE: %.5f" % mae_errors[i])
    print()
    
for i in range(30):
    row = random.randint(0, len(test_emotions))

    print("Real: " + str([test_emotions_OG.iloc[row,0], test_emotions_OG.iloc[row,1], test_emotions_OG.iloc[row,2]]))
    print("Predicted: " + str([test_emotions.iloc[row,0], test_emotions.iloc[row,1], test_emotions.iloc[row,2]]))
    print("Generated: " + str([gen_emotions.iloc[row,0], gen_emotions.iloc[row,1], gen_emotions.iloc[row,2]]))
    print()

Overall MAE: 0.2576214060728437

==PLEASURE==
MSE: 0.04190
MAE: 0.15866

==AROUSAL==
MSE: 0.21868
MAE: 0.38214

==DOMINANCE==
MSE: 0.10869
MAE: 0.23207

Real: [-0.1, -0.6, -0.15]
Predicted: [-0.11588165163993835, -0.6001495718955994, -0.15008001029491425]
Generated: [0.02680247, -0.5177072, -0.21459351]

Real: [0.05, -0.05, 0.0]
Predicted: [0.05253050476312637, -0.02385752834379673, 0.0033056570682674646]
Generated: [0.15047622, -0.26433584, -0.13881762]

Real: [0.1, 0.6, 0.4]
Predicted: [0.09773490577936172, 0.6142454743385315, 0.394778847694397]
Generated: [0.05857926, 0.15244707, 0.023716396]

Real: [-0.1, -0.6, -0.15]
Predicted: [-0.10124903172254562, -0.5726799368858337, -0.15798325836658478]
Generated: [0.011174473, -0.4907463, -0.14176269]

Real: [0.05, -0.05, 0.0]
Predicted: [0.05077561363577843, -0.040145277976989746, -0.015249746851623058]
Generated: [0.1305307, -0.22626574, -0.07997816]

Real: [0.05, -0.05, 0.0]
Predicted: [0.04962637647986412, -0.12842907011508942, 0.003484

In [56]:
# save to JSON
for i in range(len(best_regressors)):
    best_regressors[i].save_model("models/bandai/5frame/xgb/" + test_y_sets[i].columns[0] +".json")

### Predictions

In [57]:
model_p = xgb.XGBRegressor(verbosity=0)
model_p.load_model("../../emotion_classifier/model_training/models/bandai_l2p_model.json")

model_a = xgb.XGBRegressor(verbosity=0)
model_a.load_model("../../emotion_classifier/model_training/models/bandai_l2a_model.json")

model_d = xgb.XGBRegressor(verbosity=0)
model_d.load_model("../../emotion_classifier/model_training/models/bandai_l2d_model.json")

scaler = joblib.load('../../emotion_classifier/model_training/datasets/scalers/standardizers/S_BANDAI_5frame.pkl') 

In [58]:
index = 10

real_coordinates = np.asarray([test_emotions.iloc[index]])

generated_features = []
for i in range(len(models)):
    generated_features.append(best_regressors[i].predict(real_coordinates)[0])
    #print(test_y_sets[i].columns[0], " - ", generated_features[i])
    
generated_features = pd.DataFrame([generated_features], columns=["max_hand_distance",
          "avg_l_hand_hip_distance",
          "avg_r_hand_hip_distance",
          "max_stride_length",
          "avg_l_hand_chest_distance",
          "avg_r_hand_chest_distance",
          "avg_l_elbow_hip_distance",
          "avg_r_elbow_hip_distance",
          "avg_chest_pelvis_distance",
          "avg_neck_chest_distance",
          
          "avg_total_body_volume",
          "avg_lower_body_volume",
          "avg_upper_body_volume",
          
          "avg_triangle_area_hands_neck",
          "avg_triangle_area_feet_hips",
          
          "l_hand_speed",
          "r_hand_speed",
          "l_foot_speed",
          "r_foot_speed",
          "neck_speed",
          
          "l_hand_acceleration_magnitude",
          "r_hand_acceleration_magnitude",
          "l_foot_acceleration_magnitude",
          "r_foot_acceleration_magnitude",
          "neck_acceleration_magnitude",
    ])

generated_features = scaler.transform(generated_features)

y_p = model_p.predict(generated_features)
y_a = model_a.predict(generated_features)
y_d = model_d.predict(generated_features)

print('Real: %s' % np.asarray(real_coordinates))
print('Predicted: %s' % [y_p[0], y_a[0], y_d[0]])

Real: [[ 0.03395629 -0.0323584   0.00172423]]
Predicted: [0.14783467, -0.440891, -0.15377945]


In [61]:
real_coordinates = np.asarray([[0.0, 0.0, 0.0]])

generated_features = []
for i in range(len(models)):
    generated_features.append(best_regressors[i].predict(real_coordinates)[0])
    #print(test_y_sets[i].columns[0], " - ", generated_features[i])
    
generated_features = pd.DataFrame([generated_features], columns=["max_hand_distance",
          "avg_l_hand_hip_distance",
          "avg_r_hand_hip_distance",
          "max_stride_length",
          "avg_l_hand_chest_distance",
          "avg_r_hand_chest_distance",
          "avg_l_elbow_hip_distance",
          "avg_r_elbow_hip_distance",
          "avg_chest_pelvis_distance",
          "avg_neck_chest_distance",
          
          "avg_total_body_volume",
          "avg_lower_body_volume",
          "avg_upper_body_volume",
          
          "avg_triangle_area_hands_neck",
          "avg_triangle_area_feet_hips",
          
          "l_hand_speed",
          "r_hand_speed",
          "l_foot_speed",
          "r_foot_speed",
          "neck_speed",
          
          "l_hand_acceleration_magnitude",
          "r_hand_acceleration_magnitude",
          "l_foot_acceleration_magnitude",
          "r_foot_acceleration_magnitude",
          "neck_acceleration_magnitude",
    ])

generated_features = scaler.transform(generated_features)

y_p = model_p.predict(generated_features)
y_a = model_a.predict(generated_features)
y_d = model_d.predict(generated_features)

print('Real: %s' % np.asarray(real_coordinates))
print('Predicted: %s' % [y_p[0], y_a[0], y_d[0]])

Real: [[0. 0. 0.]]
Predicted: [0.117322154, -0.4578352, -0.15295118]
